# Book Recommendation System
###  [Dataset](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?resource=download)

In [225]:
import pandas as pd

books = pd.read_csv("Books.csv")
ratings = pd.read_csv("Ratings.csv")
users = pd.read_csv("Users.csv")

C:\Users\krylt\AppData\Local\Temp\ipykernel_2348\2595514696.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Books.csv")


## Подготовка данных

### Books

In [227]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [229]:
books = books.drop(['Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)
books.rename(columns={'Book-Author': 'Author', 'Year-Of-Publication': 'Year'}, inplace=True)
books.dropna(subset=['Author'], inplace=True)
books = books.drop_duplicates(subset='Book-Title')

In [231]:
books['Year'] = pd.to_numeric(books['Year'], errors='coerce')
books['Year'] = books['Year'].map(lambda x: books['Year'].median() if x == 0 else x)

books_copy = books.copy()

books['Author'] = pd.factorize(books['Author'])[0].astype(int)

In [232]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242133 entries, 0 to 271359
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ISBN        242133 non-null  object 
 1   Book-Title  242133 non-null  object 
 2   Author      242133 non-null  int32  
 3   Year        242130 non-null  float64
dtypes: float64(1), int32(1), object(2)
memory usage: 8.3+ MB


In [53]:
books.head()

,ISBN,Book-Title,Author,Year
0,0195153448,Classical Mythology,0,2002.0
1,0002005018,Clara Callan,1,2001.0
2,0060973129,Decision in Normandy,2,1991.0
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,3,1999.0
4,0393045218,The Mummies of Urumchi,4,1999.0


### Users

In [55]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [56]:
users['Location'] = pd.factorize(users['Location'])[0]

users['Age'] = users['Age'].fillna(users['Age'].median())

In [57]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  int64  
 2   Age       278858 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 6.4 MB


In [58]:
users.head()

,User-ID,Location,Age
0,1,0,32.0
1,2,1,18.0
2,3,2,32.0
3,4,3,17.0
4,5,4,32.0


### Ratings

In [63]:
ratings = ratings[ratings['Book-Rating'] != 0]
rows = ratings.shape[0]
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 433671 entries, 1 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      433671 non-null  int64 
 1   ISBN         433671 non-null  object
 2   Book-Rating  433671 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 13.2+ MB


In [64]:
ratings.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6


### Объединение датасетов

In [126]:
df = ratings.merge(users, on='User-ID').merge(books, on='ISBN')
df

,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Author,Year
0,276729,052165615X,3,4171,16.0,Help!: Level 1,89642,1999.0
1,276729,0521795028,6,4171,16.0,The Amsterdam Connection : Level 4 (Cambridge ...,30205,2001.0
2,276747,0060517794,9,410,25.0,Little Altars Everywhere,127,2003.0
3,276747,0671537458,9,410,25.0,Waiting to Exhale,1333,1995.0
4,276747,0679776818,8,410,25.0,Birdsong: A Novel of Love and War,1141,1997.0
...,...,...,...,...,...,...,...,...
328228,276704,0743211383,7,1823,32.0,Dreamcatcher,119,2001.0
328229,276704,0806917695,5,1823,32.0,Perplexing Lateral Thinking Puzzles: Scholasti...,1635,1997.0
328230,276704,1563526298,9,1823,32.0,Get Clark Smart : The Ultimate Guide for the S...,22982,2000.0
328231,276709,0515107662,10,52320,38.0,The Sherbrooke Bride (Bride Trilogy (Paperback)),452,1996.0


In [132]:
df.dropna(subset=['Year'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 328232 entries, 0 to 328232
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   User-ID      328232 non-null  int64  
 1   ISBN         328232 non-null  object 
 2   Book-Rating  328232 non-null  int64  
 3   Location     328232 non-null  int64  
 4   Age          328232 non-null  float64
 5   Book-Title   328232 non-null  object 
 6   Author       328232 non-null  int32  
 7   Year         328232 non-null  float64
dtypes: float64(2), int32(1), int64(3), object(2)
memory usage: 21.3+ MB


## Контентная фильтрация

In [134]:
from sklearn.model_selection import train_test_split

X = df[['Age', 'Location', 'Author', 'Book-Title', 'Year']]
y = df['Book-Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=22)

In [139]:
from catboost import CatBoostRegressor

catboost = CatBoostRegressor(iterations=800,
                             depth=9,
                             learning_rate=0.04,
                             verbose=100,
                             cat_features=['Location', 'Author', 'Book-Title'])

catboost.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

0:	learn: 1.8391759	test: 1.8439418	best: 1.8439418 (0)	total: 298ms	remaining: 3m 58s
100:	learn: 1.7075490	test: 1.6802459	best: 1.6802459 (100)	total: 12.5s	remaining: 1m 26s
200:	learn: 1.6949180	test: 1.6695222	best: 1.6695222 (200)	total: 28.1s	remaining: 1m 23s
300:	learn: 1.6881729	test: 1.6653915	best: 1.6653915 (300)	total: 43s	remaining: 1m 11s
400:	learn: 1.6800584	test: 1.6605012	best: 1.6605012 (400)	total: 1m 3s	remaining: 1m 3s
500:	learn: 1.6740159	test: 1.6573980	best: 1.6573980 (500)	total: 1m 23s	remaining: 50.1s
600:	learn: 1.6691389	test: 1.6556266	best: 1.6556266 (600)	total: 1m 42s	remaining: 33.8s
700:	learn: 1.6647858	test: 1.6539557	best: 1.6539557 (700)	total: 2m 1s	remaining: 17.2s
799:	learn: 1.6607174	test: 1.6529280	best: 1.6529280 (799)	total: 2m 20s	remaining: 0us

bestTest = 1.652928037
bestIteration = 799



In [140]:
from sklearn.metrics import mean_squared_error

y_pred = catboost.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

RMSE: 1.6555737146571985


C:\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [191]:
def content_recommendations(user_id, model, books, users, n_recs=10, for_hybrid=False):
    # Получаем данные о пользователе
    user_info = users[users['User-ID'] == user_id].iloc[0]
    user_location = int(user_info['Location'])
    user_age = user_info['Age']
    
    # Создаём датафрейм с данными о пользователе и всех книгах
    user_books = books.copy()
    user_books['Age'] = user_age
    user_books['Location'] = user_location
    
    # Предсказываем оценки
    predictions = model.predict(user_books[['Age', 'Location', 'Author', 'Book-Title', 'Year']])
    user_books = user_books.drop(['Age', 'Location'], axis=1)
    user_books['Predicted-Rating'] = predictions
    
    if for_hybrid:
        return user_books
    
    recommendations = user_books.sort_values(by='Predicted-Rating', ascending=False).head(n_recs)  
    return recommendations

In [193]:
recommendations = content_recommendations(user_id=276688, model=catboost, books=books_copy, users=users)
print(recommendations[['Book-Title', 'Predicted-Rating']].to_string(index=False))

                                                 Book-Title  Predicted-Rating
                                      To Kill a Mockingbird          8.471093
                              Little House in the Big Woods          8.432988
                                     How to Eat Fried Worms          8.432972
                                 On the Banks of Plum Creek          8.427980
Cat in the Hat (I Can Read It All by Myself Beginner Books)          8.390325
                Secret Warning (Hardy Boys Mystery Stories)          8.385903
                     Stuart Little 60th Anniversary Edition          8.377571
                                         The Cat in the Hat          8.376445
                                        Pippi Longstockings          8.375158
                       Cyrano De Bergerac (Bantam Classics)          8.375158


## Коллаборативная фильтрация

In [65]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 433671 entries, 1 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      433671 non-null  int64 
 1   ISBN         433671 non-null  object
 2   Book-Rating  433671 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 13.2+ MB


In [76]:
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)

# Разделение данных
trainset, testset = train_test_split(data, test_size=0.2)

# Обучение модели SVD
model = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)
model.fit(trainset)

predictions = model.test(testset) 
print('RMSE:', accuracy.rmse(predictions))

RMSE: 1.6405
RMSE: 1.6404934557320365


In [211]:
def collaborative_recommendations(user_id, n_recs=10, for_hybrid=False):
    # Создаем список книг, которые пользователь еще не оценил
    all_books = ratings['ISBN'].unique()
    rated_books = ratings[ratings['User-ID'] == user_id]['ISBN'].values
    unrated_books = [book_id for book_id in all_books if book_id not in rated_books]
    
    # Создаем список предсказанных рейтингов и преобразуем в DataFrame
    predicted_ratings = []
    
    for book_id in unrated_books:
        prediction = model.predict(user_id, book_id)
        predicted_ratings.append((book_id, prediction.est))
    
    predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=['ISBN', 'Predicted-Rating'])
    # Сортируем и выбираем топ-N книг
    top_books = predicted_ratings_df.sort_values(by='Predicted-Rating', ascending=False)
    recommend_books = top_books.merge(books, on='ISBN')
    
    if for_hybrid:
        return top_books
    
    return recommend_books[['Book-Title', 'Predicted-Rating']].head(n_recs)

In [213]:
recommendations = collaborative_recommendations(user_id=276688)
print(recommendations.to_string(index=False))

                                                      Book-Title  Predicted-Rating
           Harry Potter and the Chamber of Secrets Postcard Book          9.474604
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))          9.370764
                    My Sister's Keeper : A Novel (Picoult, Jodi)          9.361047
                                                Love You Forever          9.329964
                                        The Time Traveler's Wife          9.324130
                                                   Lonesome Dove          9.300149
                                   The Stand: Complete and Uncut          9.280658
              Harry Potter and the Order of the Phoenix (Book 5)          9.278696
                                        A Tree Grows in Brooklyn          9.274684
                                               The Da Vinci Code          9.272303


In [ ]:
users_similarity_df

## Гибридная фильтрация

In [215]:
def hybrid_recommendations(user_id, n_recs=10):
    # Рекомендаций на основе коллаборативной фильтрации
    collab_recs = collaborative_recommendations(user_id, for_hybrid=True)
    # Рекомендаций на основе контентной фильтрации
    content_recs = content_recommendations(user_id=user_id, model=catboost, books=books_copy, users=users, for_hybrid=True)
    
    # Объединяем результаты, вычисляя средний рейтинг
    hybrid_recs = pd.merge(collab_recs, content_recs, on='ISBN', how='inner', suffixes=(' (collab)', ' (content)'))
    hybrid_recs['Predicted-Rating'] = (hybrid_recs['Predicted-Rating (collab)'] + hybrid_recs['Predicted-Rating (content)']) / 2

    recommendations = hybrid_recs.sort_values(by='Predicted-Rating', ascending=False).head(n_recs)
    return recommendations

In [219]:
recommendations = hybrid_recommendations(user_id=276688)
print(recommendations[['Book-Title', 'Predicted-Rating']].to_string(index=False))

                                                      Book-Title  Predicted-Rating
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))          8.830909
                    My Sister's Keeper : A Novel (Picoult, Jodi)          8.815334
           Harry Potter and the Chamber of Secrets Postcard Book          8.801925
          The Return of the King (The Lord of the Rings, Part 3)          8.797213
                                                Love You Forever          8.787701
                                                           Night          8.785596
    The Hobbit : The Enchanting Prelude to The Lord of the Rings          8.785547
                                              The Cat in the Hat          8.782305
              Harry Potter and the Order of the Phoenix (Book 5)          8.757841
                                                   Lonesome Dove          8.757826
